In [1]:
import pandas as pd
import numpy as np

import altair as alt

import matplotlib.pyplot as plt
from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.grid_finder as grid_finder
import mpl_toolkits.axisartist.floating_axes as floating_axes

%matplotlib inline

In [2]:
def Taylor_diag(series, names):
    """
        Taylor Diagram : obs is reference data sample
        in a full diagram (0 --> npi)
        --------------------------------------------------------------------------
        Input: series     - dict with all time series (lists) to analyze  
               series[0]  - is the observation, the reference by default.
    """
    corr, std = {}, {}
    for i in series.keys():
        corr[i] = np.ma.corrcoef(series[0], series[i])[1, 0]
        std[i] = np.ma.std(series[i]) / np.ma.std(series[0])
       
    reference_value = 1  # ma.std(series[0])
    #print corr
    
    rlocs = np.concatenate((np.arange(0, -10, -0.25), [-0.95,-0.99],
                            np.arange(0,10,0.25), [0.95,0.99]))
    str_rlocs = np.concatenate((np.arange(0,10,0.25), [0.95,0.99],
                                np.arange(0,10,0.25), [0.95,0.99]))
    tlocs = np.arccos(rlocs)        # Conversion to polar angles
    grid_locator_1 = grid_finder.FixedLocator(tlocs)    # Positions
    tick_formatter_1 = grid_finder.DictFormatter(
        dict(zip(tlocs, map(str, rlocs))))

    str_locs2 = np.arange(-10, 11, 0.5)
    tlocs2 =  np.arange(-10, 11, 0.5)      
       
    grid_locator_2 = grid_finder.FixedLocator(tlocs2)  # ALEKS: THIS IS NOT USED ANYWHERE, CHECK
    tick_formatter_2 = grid_finder.DictFormatter(
        dict(zip(tlocs2, map(str, str_locs2))))
    
    base_transform = PolarAxes.PolarTransform()
    
    smin = 0
    smax = 2.5

    grid_helper = floating_axes.GridHelperCurveLinear(
        base_transform,
        extremes=(0, np.pi, smin, smax),  # 1st quadrant
        grid_locator1=grid_locator_1,
        #grid_locator2=grid_locator_2,
        tick_formatter1=tick_formatter_1,
        tick_formatter2=tick_formatter_2)
    
    fig = plt.figure(figsize=(10,5), dpi=100)
    ax = floating_axes.FloatingSubplot(fig, 111, grid_helper=grid_helper)

    fig.add_subplot(ax)
    ax.axis["top"].set_axis_direction("bottom") 
    ax.axis["top"].toggle(ticklabels=True, label=True)
    ax.axis["top"].major_ticklabels.set_axis_direction("top")
    ax.axis["top"].label.set_axis_direction("top")
    ax.axis["top"].label.set_text("Correlation Coefficient")

    ax.axis["left"].set_axis_direction("bottom") 
    ax.axis["left"].label.set_text("Standard Deviation")

    ax.axis["right"].set_axis_direction("top") 
    ax.axis["right"].toggle(ticklabels=True, label=True)
    ax.axis["right"].set_visible(True)
    ax.axis["right"].major_ticklabels.set_axis_direction("bottom")
    #ax.axis["right"].label.set_text("Standard Deviation")
    
    ax.axis["bottom"].set_visible(False) 
    ax.grid(True)
    ax = ax.get_aux_axes(base_transform)

    t = np.linspace(0, np.pi)
    r = np.zeros_like(t) + reference_value
    ax.plot(t, r, 'k--', label='_')

    rs, ts = np.meshgrid(np.linspace(smin, smax), np.linspace(0, np.pi))

    rms = np.sqrt(reference_value**2 + rs**2 - 2*reference_value*rs*np.cos(ts))
    CS =ax.contour(ts, rs, rms, cmap=plt.matplotlib.cm.bone)
    plt.clabel(CS, inline=1, fontsize=10)
    
    ax.plot(np.arccos(0.9999), reference_value, 'k', marker='*', ls='', ms=10)
    aux = range(1, len(corr))
    #del aux[reference_value]

    colors = plt.matplotlib.cm.jet(np.linspace(0, 1, len(corr)))
    
    for i in aux:
        ax.plot(np.arccos(corr[i]), std[i], c=colors[i], alpha=0.7, ms=15,
                marker='o', label=" %s" % names[i])
        ax.text(np.arccos(corr[i]), std[i], "%s" % i, fontsize=12)
    plt.legend(bbox_to_anchor=(1.5, 1), prop=dict(size='large'), loc='best')
    plt.plot()  # ALEKS: Changed from savefig("example.png", dpi=300)
    return


In [ ]:
z = np.linspace(0,2048,2048)
x = np.sin(50*np.pi*z)
y = np.cos(50*np.pi*z)
x2 = 2*np.sin(50*np.pi*z)

plt.plot(x, 'k--')

In [ ]:
plt.plot(x2)

In [ ]:
plt.plot(y)

In [ ]:


series ={}
series[0] = x  # the first term will be always the reference_valueerence signal 
series[1]= x2
series[2]= y



In [ ]:
Taylor_diag(series,['x','x2','y'])

In [52]:
source = pd.DataFrame({"values": [12, 23, 47, 6, 52, 19]})

base = alt.Chart(source).encode(
    theta=alt.Theta("values:Q", stack=True),
    radius=alt.Radius("values", scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color="values:N",
)

c1 = base.mark_arc(innerRadius=20, stroke="#fff")

c2 = base.mark_text(radiusOffset=10).encode(text="values:Q")

c1 + c2

alt.LayerChart(...)

In [67]:
df_new = pd.DataFrame(dict(
    {
        'theta_deg': range(10, 190, 10),
        'theta_2_deg': range(0, 180, 10),
        #'r': range(0, 180, 10),
    }))

df_new['theta_rad'] = np.pi * (df_new['theta_deg']/180)
df_new['theta_2_rad'] = np.pi * (df_new['theta_2_deg']/180)

df_new

,theta_deg,theta_2_deg,theta_rad,theta_2_rad
0,10,0,0.174533,0.000000
1,20,10,0.349066,0.174533
2,30,20,0.523599,0.349066
3,40,30,0.698132,0.523599
4,50,40,0.872665,0.698132
5,60,50,1.047198,0.872665
6,70,60,1.221730,1.047198
7,80,70,1.396263,1.221730
8,90,80,1.570796,1.396263
9,100,90,1.745329,1.570796


In [71]:
float_start_circle = -np.pi*0.5
float_end_circle = np.pi*0.5

chart_test = alt.Chart(df_new).mark_arc(color='grey', opacity=0.8).encode(
    theta=alt.Theta('theta_rad:Q', scale=alt.Scale(type="linear", rangeMax=float_end_circle, rangeMin=float_start_circle)),
    theta2=alt.Theta2('theta_2_rad:Q'),
    
)

chart_test


alt.Chart(...)